In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import copy
import math

In [2]:
df = pd.read_csv("C:\\Users\\bimalgupta\\Desktop\\OTIS\\data\\final_df2_new1.csv",)

In [3]:
import copy
data_df = copy.copy(df)
df = data_df.iloc[0:,1:]

In [4]:
df['Date'] = pd.to_datetime(df['Date'],errors='raise') 

In [5]:
# leat's crate one feature year
df["yr"] =  pd.DatetimeIndex(df['Date']).year

In [6]:
df.head()

,Symbol,Date,Close Price,yr
0,ADANIPORTS,2007-11-27,962.90,2007
1,ADANIPORTS,2007-11-28,893.90,2007
2,ADANIPORTS,2007-11-29,884.20,2007
3,ADANIPORTS,2007-11-30,921.55,2007
4,ADANIPORTS,2007-03-12,969.30,2007


In [7]:
# calculate return
df['Return']= abs(df['Close Price']/df['Close Price'].shift()-1)

In [8]:
df['Return1']= df['Close Price']/df['Close Price'].shift()-1

In [9]:
# lets check duplicate in the data
print("Toatl duplicate data in the data set:",df.duplicated(subset=None, keep='first').sum())
duplicateRowsDF = df[df.duplicated(subset=None, keep='first')]
print("Name of duplicate row:\n",duplicateRowsDF)
# drop duplicate row
df.dropna(axis=0,inplace=True)

Toatl duplicate data in the data set: 0
Name of duplicate row:
 Empty DataFrame
Columns: [Symbol, Date, Close Price, yr, Return, Return1]
Index: []


In [10]:
# copy the data
np.random.seed(5)
data = copy.copy(df)

In [11]:
data.head()

,Symbol,Date,Close Price,yr,Return,Return1
1,ADANIPORTS,2007-11-28,893.90,2007,0.071659,-0.071659
2,ADANIPORTS,2007-11-29,884.20,2007,0.010851,-0.010851
3,ADANIPORTS,2007-11-30,921.55,2007,0.042242,0.042242
4,ADANIPORTS,2007-03-12,969.30,2007,0.051815,0.051815
5,ADANIPORTS,2007-04-12,1041.45,2007,0.074435,0.074435


In [12]:
data.shape

(218568, 6)

In [13]:
# drop 1st row of the each company 
def mask_first(x):
    result = np.ones_like(x)
    result[0] = 0
    return result

mask = data.groupby('Symbol')['Symbol'].transform(mask_first).astype(bool)
data=data.loc[mask]

In [14]:
data.shape

(218518, 6)

data.head()

# Create Data Dict 
     Data Dictionary of the each comapny

In [15]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
Company_name=list(data.Symbol.unique())
import xlsxwriter


In [16]:
data_disc = []
data_name = []
for row in Company_name:
    data_as = data[data['Symbol']==row]
    print('='*50+'Data information of ' + str(row)+'='*50)
    print(data_as.info(),'\n')
    print('='*50+"Data discription of "+ str(row)+'='*50 )
    print(str(row)+ ':\n' ,data_as.drop('yr',axis=1).describe())
    data_disc.append(data_as.drop('yr',axis=1).describe()) 
    data_name.append(row)

==================================================Data information of ADANIPORTS==================================================
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2823 entries, 2 to 2824
Data columns (total 6 columns):
Symbol         2823 non-null object
Date           2823 non-null datetime64[ns]
Close Price    2823 non-null float64
yr             2823 non-null int64
Return         2823 non-null float64
Return1        2823 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 154.4+ KB
None 

==================================================Data discription of ADANIPORTS==================================================
ADANIPORTS:
        Close Price       Return      Return1
count  2823.000000  2823.000000  2823.000000
mean    184.661456     0.019600     0.000309
std     256.916229     0.026130     0.032665
min      21.600000     0.000000    -0.960430
25%      30.885000     0.006040    -0.014248
50%      61.300000     0.014282  

In [17]:
writer = pd.ExcelWriter('C:\\Users\\bimalgupta\\Desktop\\OTIS\\Code\\discriptive_data_dict.xlsx', engine='xlsxwriter')
for name,df in zip(data_name,data_disc):
    df.to_excel(writer, sheet_name='Data dicription '+name)
writer.save()

In [18]:
B_data = copy.copy(data)

In [19]:
#total number of bins
b= [0,0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,2]

In [20]:
l = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
B_data['binned']=pd.cut(x=B_data['Return'], bins=b,labels=l)

In [21]:
B_data.head()

,Symbol,Date,Close Price,yr,Return,Return1,binned
2,ADANIPORTS,2007-11-29,884.20,2007,0.010851,-0.010851,2
3,ADANIPORTS,2007-11-30,921.55,2007,0.042242,0.042242,2
4,ADANIPORTS,2007-03-12,969.30,2007,0.051815,0.051815,3
5,ADANIPORTS,2007-04-12,1041.45,2007,0.074435,0.074435,3
6,ADANIPORTS,2007-05-12,1082.45,2007,0.039368,0.039368,2


In [22]:
# create the bins for return
pivot_n= B_data.pivot_table(index=['Symbol'],values=["Return"],columns=['binned'],aggfunc=lambda x: len(x.unique()))
pivot_n.fillna(0,inplace=True)

In [23]:
# total 
pivot_n.columns = [col[1] for col in pivot_n.columns]
pivot_n['tot'] = pivot_n[list(pivot_n)].sum(axis=1)

In [24]:
pivot_n.head()

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,tot
Symbol,,,,,,,,,,,,,,,,,,,,,
ADANIPORTS,1063.0,1549.0,166.0,12.0,3.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2796.0
ASIANPAINT,2776.0,2145.0,98.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5025.0
AXISBANK,1868.0,2662.0,352.0,16.0,8.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4912.0
BAJAJ-AUTO,1386.0,1253.0,53.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2699.0
BAJAJFINSV,1220.0,1350.0,115.0,5.0,5.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2696.0


In [25]:
pivot_n.tail()

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,tot
Symbol,,,,,,,,,,,,,,,,,,,,,
UPL,1471.0,2090.0,180.0,9.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3759.0
VEDL,1579.0,2834.0,516.0,53.0,8.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4994.0
WIPRO,2168.0,2479.0,338.0,36.0,9.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,5038.0
YESBANK,1259.0,1877.0,223.0,22.0,1.0,4.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3390.0
ZEEL,1786.0,2723.0,446.0,40.0,15.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5017.0


In [26]:
# z_score should be by each comapny 

In [27]:
# Z_score of each comapny
# create the bins for Z-Score
dict_company = {} # 
#data_frame_list = []
new_df = pd.DataFrame()
from scipy import stats
for row in Company_name:
    B_data1 = B_data.loc[B_data['Symbol']==row]
    B_data1['Return_Zscore'] = stats.zscore(B_data1['Return1'])
    #B_data1['Return_Zscore'] =return_zscore # B_data1 contain Z_score of each comapny
    new_df=new_df.append(B_data1)
    #dict_company[row] = (return_zscore.min(), return_zscore.max())

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [28]:
# let's create the bins for the Z_score
B= [-50,-10,-5,0,5,10,15,] # bins
L = [i for i in range(1,len(B))] # label

In [29]:
len(B), len(L)

(7, 6)

In [30]:
new_df['binned_Zscore']= pd.cut(x=new_df['Return_Zscore'], bins=B,labels=L)

In [31]:
new_df.head()

,Symbol,Date,Close Price,yr,Return,Return1,binned,Return_Zscore,binned_Zscore
2,ADANIPORTS,2007-11-29,884.20,2007,0.010851,-0.010851,2,-0.341722,3
3,ADANIPORTS,2007-11-30,921.55,2007,0.042242,0.042242,2,1.283939,4
4,ADANIPORTS,2007-03-12,969.30,2007,0.051815,0.051815,3,1.577066,4
5,ADANIPORTS,2007-04-12,1041.45,2007,0.074435,0.074435,3,2.269681,4
6,ADANIPORTS,2007-05-12,1082.45,2007,0.039368,0.039368,2,1.195959,4


In [32]:
pivot_z= new_df.pivot_table(index=['Symbol'],values=['Return_Zscore'],columns=['binned_Zscore'],aggfunc=lambda x: len(x.unique()))

In [33]:
pivot_z.fillna(0,inplace=True)

In [34]:
pivot_z.head()

Return_Zscore                               
binned_Zscore             1    2       3       4    5    6
Symbol                                                    
ADANIPORTS              1.0  1.0  1426.0  1368.0  4.0  0.0
ASIANPAINT              3.0  1.0  2615.0  2411.0  0.0  0.0
AXISBANK                1.0  1.0  2583.0  2342.0  8.0  0.0
BAJAJ-AUTO              1.0  2.0  1400.0  1295.0  2.0  0.0
BAJAJFINSV              0.0  2.0  1433.0  1259.0  4.0  0.0

In [35]:
# sum of each company return_zscore count
pivot_z.columns = [col[1] for col in pivot_z.columns]
pivot_z['Total'] = pivot_z[list(pivot_z)].sum(axis=1)

In [36]:
pivot_z.head()

,1,2,3,4,5,6,Total
Symbol,,,,,,,
ADANIPORTS,1.0,1.0,1426.0,1368.0,4.0,0.0,2800.0
ASIANPAINT,3.0,1.0,2615.0,2411.0,0.0,0.0,5030.0
AXISBANK,1.0,1.0,2583.0,2342.0,8.0,0.0,4935.0
BAJAJ-AUTO,1.0,2.0,1400.0,1295.0,2.0,0.0,2700.0
BAJAJFINSV,0.0,2.0,1433.0,1259.0,4.0,0.0,2698.0


In [37]:
# Number of count should be same as in pivot table
new_df.groupby('Symbol').count().head(10)

,Date,Close Price,yr,Return,Return1,binned,Return_Zscore,binned_Zscore
Symbol,,,,,,,,
ADANIPORTS,2823,2823,2823,2823,2823,2807,2823,2823
ASIANPAINT,5062,5062,5062,5062,5062,5034,5062,5062
AXISBANK,5062,5062,5062,5062,5062,4999,5062,5062
BAJAJ-AUTO,2704,2704,2704,2704,2704,2699,2704,2704
BAJAJFINSV,2703,2703,2703,2703,2703,2699,2703,2703
BAJFINANCE,4994,4994,4994,4994,4994,4905,4994,4994
BHARTIARTL,4276,4276,4276,4276,4276,4250,4276,4276
BPCL,5062,5062,5062,5062,5062,5049,5062,5062
BRITANNIA,5061,5061,5061,5061,5061,5042,5061,5061


In [38]:
pivot_z.shape

(50, 7)

In [39]:
# lets save in the excel format
writer1 = pd.ExcelWriter('C:\\Users\\bimalgupta\\Desktop\\OTIS\\Code\\pivot_n_Z.xlsx', engine='xlsxwriter')
pivot_n.to_excel(writer1, sheet_name='pivot_for_return')
pivot_z.to_excel(writer1, sheet_name='pivot_for_Zscore')
writer1.save()

In [41]:
#now create the pivot table for each of the comapny
Compnay_name=[]
r_large_pivot = []
z_large_pivot =[]
for row in Company_name:
    data_as1 = new_df[new_df['Symbol']==row]
    #print('='*40+' nlarge of Each company ' + str(row)+'='*40)
    
    B_data2=data_as1.nlargest(15, "Return")
    
    pivot_Z_large = B_data2.pivot_table(index=['Symbol'],values=['Return_Zscore'],columns=['binned_Zscore'],aggfunc=lambda x: len(x.unique()))
    
    pivot_Z_large.columns = [col[1] for col in pivot_Z_large.columns]
    pivot_Z_large['Total'] = pivot_Z_large[list(pivot_Z_large)].sum(axis=1)
    
    print(pivot_Z_large)
    print('-'*100, '\n')
    
    pivot_n_large= B_data2.pivot_table(index=['Symbol'],values=['Return'],columns=['binned'],aggfunc=lambda x: len(x.unique()))
    pivot_n_large.columns = [col[1] for col in pivot_n_large.columns]
    pivot_n_large['Total_n'] = pivot_n_large[list(pivot_n_large)].sum(axis=1)
    print('\n',pivot_n_large)
    print('='*100, '\n')
    Compnay_name.append(row)
    z_large_pivot.append(pivot_Z_large)  
    r_large_pivot.append(pivot_n_large)


              1    2    3    4    5  Total
Symbol                                    
ADANIPORTS  1.0  1.0  5.0  4.0  4.0   15.0
---------------------------------------------------------------------------------------------------- 


               4    5    6   20  Total_n
Symbol                                 
ADANIPORTS  9.0  3.0  2.0  1.0     15.0

              1    2    3    4  Total
Symbol                               
ASIANPAINT  3.0  1.0  2.0  9.0   15.0
---------------------------------------------------------------------------------------------------- 


               3    4   13   14   20  Total_n
Symbol                                      
ASIANPAINT  9.0  3.0  1.0  1.0  1.0     15.0

            1    2    3    4    5  Total
Symbol                                  
AXISBANK  1.0  1.0  2.0  3.0  8.0   15.0
---------------------------------------------------------------------------------------------------- 


             4    5    6    7   20  Total_n
Symbol             

In [42]:
writer3 = pd.ExcelWriter('C:\\Users\\bimalgupta\\Desktop\\OTIS\\Code\\n_large_of each_comapy.xlsx', engine='xlsxwriter')
writer4 = pd.ExcelWriter('C:\\Users\\bimalgupta\\Desktop\\OTIS\\Code\\n_large_zscore_each_company.xlsx', engine='xlsxwriter')

for name,df1,df2 in zip(Compnay_name,r_large_pivot,z_large_pivot):
    
    df1.to_excel(writer3, sheet_name='pivot of n large '+ str(name))
    df2.to_excel(writer4, sheet_name='pivotof_n_Zscore '+ str(name))

writer3.save()
writer4.close() 